<a href="https://colab.research.google.com/github/Anatasia0903/CSassignment2/blob/main/220244.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [52]:
pip install mixed-naive-bayes

In [ ]:
pip install scikit-learn

In [53]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.naive_bayes import GaussianNB
from mixed_naive_bayes import MixedNB
from sklearn.preprocessing import StandardScaler
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, roc_auc_score, confusion_matrix

In [54]:
dataset = pd.read_csv('travel-insurance.csv')

In [55]:
print(dataset.describe())
print(dataset.info())

        Unnamed: 0          Age  AnnualIncome  FamilyMembers  ChronicDiseases  \
count  1987.000000  1987.000000  1.987000e+03    1987.000000      1987.000000   
mean    993.000000    29.650226  9.327630e+05       4.752894         0.277806   
std     573.741812     2.913308  3.768557e+05       1.609650         0.448030   
min       0.000000    25.000000  3.000000e+05       2.000000         0.000000   
25%     496.500000    28.000000  6.000000e+05       4.000000         0.000000   
50%     993.000000    29.000000  9.000000e+05       5.000000         0.000000   
75%    1489.500000    32.000000  1.250000e+06       6.000000         1.000000   
max    1986.000000    35.000000  1.800000e+06       9.000000         1.000000   

       TravelInsurance  
count      1987.000000  
mean          0.357323  
std           0.479332  
min           0.000000  
25%           0.000000  
50%           0.000000  
75%           1.000000  
max           1.000000  
<class 'pandas.core.frame.DataFrame'>
RangeIn

In [56]:
X = dataset.drop('TravelInsurance', axis=1)
y = dataset['TravelInsurance']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)
X_val, X_test, y_val, y_test = train_test_split(X_test, y_test, test_size=0.5, random_state=42)

In [57]:
X_train = pd.DataFrame(X_train, columns=X.columns)
X_val = pd.DataFrame(X_val, columns=X.columns)
X_test = pd.DataFrame(X_test, columns=X.columns)


In [58]:
categorical_columns = ['Employment Type', 'GraduateOrNot', 'FrequentFlyer', 'EverTravelledAbroad']


In [59]:
preprocessor = ColumnTransformer(transformers=[('encoder', OneHotEncoder(drop='first'), categorical_columns)],
                                 remainder='passthrough')

In [60]:
X_train = preprocessor.fit_transform(X_train)
X_val = preprocessor.transform(X_val)
X_test = preprocessor.transform(X_test)

In [61]:
from sklearn.model_selection import GridSearchCV, StratifiedKFold

param_grid = {
    'var_smoothing': [0.1, 0.5, 1.0, 2.0, 5.0]
}

gnb = GaussianNB()
grid_search = GridSearchCV(gnb, param_grid, cv=StratifiedKFold(5), scoring='neg_mean_squared_error')
grid_search.fit(X_val, y_val)

GridSearchCV(cv=StratifiedKFold(n_splits=5, random_state=None, shuffle=False),
             estimator=GaussianNB(),
             param_grid={'var_smoothing': [0.1, 0.5, 1.0, 2.0, 5.0]},
             scoring='neg_mean_squared_error')

In [62]:
print("Best hyperparameters:", grid_search.best_params_)
print("Best score:", grid_search.best_score_)

Best hyperparameters: {'var_smoothing': 0.5}
Best score: -0.19468926553672317


In [63]:
gnb = GaussianNB(var_smoothing=grid_search.best_params_['var_smoothing'])
gnb.fit(X_train, y_train)

GaussianNB(var_smoothing=0.5)

In [64]:
from mixed_naive_bayes import MixedNB

mnb = MixedNB()
mnb.fit(X_train, y_train)

MixedNB(alpha=0.5, var_smoothing=1e-09)

In [65]:
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, roc_auc_score, confusion_matrix, cohen_kappa_score

y_pred_gnb = gnb.predict(X_test)
y_pred_mnb = mnb.predict(X_test)

print("GaussianNB:")
print("Accuracy:", accuracy_score(y_test, y_pred_gnb))
print("Precision:", precision_score(y_test, y_pred_gnb, average='weighted'))
print("Recall:", recall_score(y_test, y_pred_gnb, average='weighted'))
print("F1-score:", f1_score(y_test, y_pred_gnb, average='weighted'))
print("ROC AUC:", roc_auc_score(y_test, gnb.predict_proba(X_test)[:, 1]))
print("Confusion Matrix:\n", confusion_matrix(y_test, y_pred_gnb))
print("Cohen's Kappa Score:", cohen_kappa_score(y_test, y_pred_gnb))

print("\nMixedNB:")
print("Accuracy:", accuracy_score(y_test, y_pred_mnb))
print("Precision:", precision_score(y_test, y_pred_mnb, average='weighted'))
print("Recall:", recall_score(y_test, y_pred_mnb, average='weighted'))
print("F1-score:", f1_score(y_test, y_pred_mnb, average='weighted'))
print("ROC AUC:", roc_auc_score(y_test, mnb.predict_proba(X_test)[:, 1]))
print("Confusion Matrix:\n", confusion_matrix(y_test, y_pred_mnb))
print("Cohen's Kappa Score:", cohen_kappa_score(y_test, y_pred_mnb))

GaussianNB:
Accuracy: 0.7324414715719063
Precision: 0.7738069485494283
Recall: 0.7324414715719063
F1-score: 0.6935990758536499
ROC AUC: 0.6866971167570654
Confusion Matrix:
 [[181   5]
 [ 75  38]]
Cohen's Kappa Score: 0.3522179494123383

MixedNB:
Accuracy: 0.725752508361204
Precision: 0.725539237070428
Recall: 0.725752508361204
F1-score: 0.7071051857841156
ROC AUC: 0.6861261775620897
Confusion Matrix:
 [[167  19]
 [ 63  50]]
Cohen's Kappa Score: 0.3684834123222749
